# Table of Contents
1. dataset 준비
2. image augmentation
3. train, val.txt 파일 만들기
4. data.yaml 파일 만들기
5. 사용할 모델의 yaml파일 변경하기
6. YOLO v5 train
7. YOLO v5 detection

In [ ]:
# 구글코랩 사용 시 설치 필요 (세션 시작때마다 해주어야 함)
!pip install -U PyYAML

In [ ]:
# 필요모듈 import
import numpy as np
import pandas as pd
from glob import glob
import json
import os
import yaml
import random
import numpy as np
import shutil
import cv2
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
from cv2 import imshow
from IPython.display import display
from google.colab.patches import cv2_imshow
import cv2
import json

## 1. dataset 준비

In [ ]:
# 이미지 데이터 셋
img_ls = glob('/content/drive/MyDrive/Dataset/images/train/*.jpg')
img_ls

In [ ]:
# label 데이터셋
json_ls = glob('/content/drive/MyDrive/Dataset/labels/train/*.json')
json_ls

In [ ]:
# json으로 저장되어있던 labels file을 txt파일로 변경
# json에서는 0-1사이의 값으로 지정되지 않아서 imgaug에서 요구하는 input형식으로 좌표를 변경함
for filename in json_ls:
    with open(filename, 'r') as file:
        json_data = json.load(file)

    with open(filename.split('.')[-2] +'.txt', 'w') as f2:
        x_min = json_data['mark'][0]['coordinates'][0][0]
        y_min = json_data['mark'][0]['coordinates'][0][1]
        x_max = json_data['mark'][0]['coordinates'][2][0]
        y_max = json_data['mark'][0]['coordinates'][2][1]
        # imgaug.augmentables.bbs에서 요구하는 좌표의 형태
        b = [0, x_min, y_max, x_max, y_min]
        f2.write(" ".join(map(str, b)))
        # 좌표를 만든 json파일 삭제하지 않음 (augmentation 후 변경되지 않은 좌표를 추가적으로 변경할 때 사용)
        # os.remove(filename)

In [ ]:
json_val_ls = glob('/content/drive/MyDrive/Dataset/labels/val/*.json')
json_val_ls

In [ ]:
# json으로 저장되어있던 labels file을 txt파일로 변경
# validation 파일은 별도로 augmentation을 진행하지 않기때문에 YOLO에서 요구하는 좌표로 바로 만들면 됨
for filename in json_val_ls:
    with open(filename, 'r') as file:
        json_data = json.load(file)

    with open(filename.split('.')[-2] +'.txt', 'w') as f:
        x = round(((json_data['mark'][0]['coordinates'][0][0]/1920) + (json_data['mark'][0]['coordinates'][1][0]/1920))/2, 3)
        y = round(((json_data['mark'][0]['coordinates'][0][1]/1080) + (json_data['mark'][0]['coordinates'][2][1]/1080))/2, 3)
        w = round((np.abs(json_data['mark'][0]['coordinates'][1][0]-json_data['mark'][0]['coordinates'][0][0]))/1920, 3)
        h = round((np.abs(json_data['mark'][0]['coordinates'][2][1]-json_data['mark'][0]['coordinates'][0][1]))/1080, 3)
        a = [0, x, y, w, h]
        f.write(" ".join(map(str, a)))
        # 좌표를 만든 json파일 삭제
        os.remove(filename)

## 2. image augmentation

In [ ]:
# augmentation 필터 설정하기
seq1 = iaa.Affine(scale={'x':(0.5, 1.5), 'y':(0.5, 1.5)}) # 늘리기
seq2 = iaa.Affine(translate_percent={'x': (-0.2, 0.2), 'y':(-0.2, 0.2)}) # 옆으로 밀기
seq3 = iaa.Affine(translate_px={"x": (-20, 20), "y": (-20, 20)}) # 위아래로 늘리기
seq4 = iaa.Affine(rotate=(-45, 45)) # 사진 45도 돌리기
seq5 = iaa.Affine(shear=(-16, 16)) # 대각선으로 늘리기
# 밝기 변화 + 좌우반전
seq7= iaa.Sequential([
                    iaa.Multiply((1.2, 1.5)), 
                    iaa.Fliplr(1.0) 
                    ])

seq8 = iaa.Grayscale(alpha=1.0) # 회색

# dropout, 픽셀 조정
seq9 = iaa.Sequential([iaa.Dropout((0.05, 0.1), per_channel=0.5),
                    iaa.Multiply((0.3, 1.5)),
                    iaa.ChannelShuffle(p=1.0)])

seq10 = iaa.GaussianBlur(sigma=1.5) # 흐리게

seq11 = iaa.Rot90(1) # 90도 회전

In [ ]:
img_ls

In [ ]:
ia_bounding_boxes = []
for j, img in list(enumerate(img_ls)): 
    label_path = '/content/drive/MyDrive/Dataset/labels/train/'
    img_path = '/content/drive/MyDrive/Dataset/images/train/'
    image = cv2.imread(img)
    with open(label_path + img.split('/')[-1].split('.')[0]+'.txt', 'r') as f:
        data = f.readline()
        ls = data.split(' ')
        ia_bounding_boxes.append(BoundingBox(x1=float(ls[1]), y1=float(ls[2]), x2=float(ls[3]), y2=float(ls[4])))
        bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)
    
        f.close()

    ls = [seq1, seq2, seq3, seq4, seq5, seq7, seq8, seq9, seq10, seq11]

    for i, seq in list(enumerate(ls)):
        seq_det = seq.to_deterministic()
        image_aug = seq_det.augment_images([image])[0]
        bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

        # image_aug: 이미지 저장
        cv2.imwrite(img_path+img.split('/')[-1].split('.')[0] + '_'+ str(i) + '.jpg', image_aug)

        # textfile 복사해서 textfile 저장
        shutil.copy(label_path+img.split('/')[-1].split('.')[0] + '.txt', label_path+img.split('/')[-1].split('.')[0] + '_'+ str(i) + '.txt')
        name = label_path+img.split('/')[-1].split('.')[0] + '_' + str(i) + '.txt'

        # textfile 열어서 사이즈로 나눠서 좌표 구해주기
        with open(name, 'w') as f2:
            x = bbs_aug.bounding_boxes[j].center_x/1920
            y = bbs_aug.bounding_boxes[j].center_y/1080
            w = bbs_aug.bounding_boxes[j].width/1920
            h = bbs_aug.bounding_boxes[j].height/1080
            cl = 0
            b = [cl, x, y, w, h]
            f2.write(" ".join(map(str, b)))

In [ ]:
# augmentation 하지 않은 원본 좌표 yolo에 맞게 변경하기
json_origin = glob('/content/drive/MyDrive/Dataset/labels/train/*.json')
json_origin

In [ ]:
# json으로 저장되어있던 labels file을 txt파일로 변경
for filename in json_origin:
    with open(filename, 'r') as file:
        json_data = json.load(file)

    with open(filename.split('.')[-2] +'.txt', 'w') as f:
        x = round(((json_data['mark'][0]['coordinates'][0][0]/1920) + (json_data['mark'][0]['coordinates'][1][0]/1920))/2, 3)
        y = round(((json_data['mark'][0]['coordinates'][0][1]/1080) + (json_data['mark'][0]['coordinates'][2][1]/1080))/2, 3)
        w = round((np.abs(json_data['mark'][0]['coordinates'][1][0]-json_data['mark'][0]['coordinates'][0][0]))/1920, 3)
        h = round((np.abs(json_data['mark'][0]['coordinates'][2][1]-json_data['mark'][0]['coordinates'][0][1]))/1080, 3)
        a = [0, x, y, w, h]
        f.write(" ".join(map(str, a)))
        # 좌표를 만든 json파일 삭제
        # os.remove(filename)

## 3. train, val.txt 파일 만들기

In [ ]:
train_ls = glob('/content/drive/MyDrive/Dataset/images/train/*.jpg')
train_ls

In [ ]:
val_ls = glob('/content/drive/MyDrive/Dataset/images/val/*.jpg')
val_ls

In [ ]:
# txt파일 만들기
with open('/content/drive/MyDrive/Dataset/train.txt', 'w') as f:
    f.write('\n'.join(train_ls) + '\n')

with open('/content/drive/MyDrive/Dataset/val.txt', 'w') as f:
    f.write('\n'.join(val_ls) + '\n')

## 4. data.yaml 파일 만들기

In [ ]:
# yaml파일 열기
with open ('/content/drive/MyDrive/Dataset/data.yaml','r') as f:
    data = yaml.load(f)

In [ ]:
print(data)

In [ ]:
# 클래스 이름은 person으로 변경하고 nc 클래스의 수는 1로 변경
data['names'] = 'person'
data['nc'] = 1
data['train'] = '/content/drive/MyDrive/Dataset/train.txt'
data['val'] = '/content/drive/MyDrive/Dataset/val.txt'

In [ ]:
with open('/content/drive/MyDrive/Data/data.yaml', 'w') as f:
    yaml.dump(data, f)

In [ ]:
print(data)

## 5. 사용할 모델의 yaml파일 변경하기
- './yolov5/models/yolov5s.yaml' yaml파일 열기 (구글코랩에서는 바로 열고 수정 가능/ 로컬에서 사용 시, 위의 방법처럼 with open으로 열어서 수정필요)
- parameters에 nc 1로 수정 (실제 detection 하고자 하는 class의 수로 변경)

## 6. YOLO v5 train

- yolo v5가 클론되어 있는 곳으로 경로 이동

In [ ]:
cd /content/drive/MyDrive/yolov5

In [ ]:
# train.py 실행
# 설정값: --data, --epochs, --cfg, -- weights, --batch-size, --name
# data는 data.yaml파일 경로, cfg는 돌리고자 하는 모델의 yaml파일 경로(현재는 v5s버전으로 설정), weights는 없어도 되고, 사전학습된 weight가 있다면 사용가능 .pt파일 경로
!python train.py --data '/content/drive/MyDrive/Dataset/data.yaml' --epochs 1000 --cfg '/content/drive/MyDrive/yolov5/models/yolov5s.yaml' --weights '/content/drive/MyDrive/yolov5/runs/train/yolov5s_results_1218/weights/best.pt' --batch-size 64 --name yolo_v5_customdata

In [ ]:
# train.py 실행
# 설정값: --data, --epochs, --cfg, -- weights, --batch-size, --name
# data는 data.yaml파일 경로, cfg는 돌리고자 하는 모델의 yaml파일 경로(현재는 v5s버전으로 설정), weights는 없어도 되고, 사전학습된 weight가 있다면 사용가능 .pt파일 경로
!python train.py --data '/content/drive/MyDrive/Dataset/data.yaml' --epochs 1000 --cfg '/content/drive/MyDrive/yolov5/models/yolov5s.yaml' --weights '' --batch-size 64 --name yolo_v5_git

## 7. YOLO v5 detection

In [ ]:
# 3번 영상에 detection
!python detect.py --weights "/content/drive/MyDrive/yolov5/runs/train/yolo_v5_customdata6/weights/best.pt" --conf 0.3 --source "/content/drive/MyDrive/Dataset/03.mp4"

In [ ]:
cd /content/drive/MyDrive/yolov5

In [ ]:
# 2번 영상에 detection
!python detect.py --weights "/content/drive/MyDrive/yolov5/runs/train/yolo_v5_git/weights/best.pt" --conf 0.3 --source "/content/drive/MyDrive/Dataset/02.mp4"